In [1]:
!pip install flask
!pip install flask-cors
!pip install gevent
!pip install pyngrok
!pip install nest-asyncio
!pip install simpletransformers==0.61.13
!pip uninstall transformers
!pip install transformers==4.10.0
!git clone https://github.com/GoldenRMT/WikiSearch.git
!pip install googledrivedownloader

     |████████████████████████████████| 5.8 MB 5.5 MB/s 
     |████████████████████████████████| 251 kB 35.8 MB/s 
     |████████████████████████████████| 745 kB 5.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=904e9fe7294359cd237821ab8f974acc8cdbd389128dbc81415f56379c0f37c5
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
     |████████████████████████████████| 221 kB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 38.8 MB/s 
     |████████████████████████████████| 124 kB 47.3 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 287 kB 47.4 MB/s 
     |████████████████████████████████| 2.9 MB 37.9 MB/s 
     |████████████████████████████████| 3.3 MB 43.7 MB/s 
     |████████████████████████████████| 8.3 MB 10.4 MB/s 
     |████████████████████████████████| 1.7 MB 41.0 MB/s 
     |█████████████████

Found existing installation: transformers 4.11.3
Uninstalling transformers-4.11.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.11.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.11.3
     |████████████████████████████████| 2.8 MB 5.4 MB/s 
Cloning into 'WikiSearch'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 99 (delta 35), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='13Nuwm7BV-4RXI9JqjPTDE9rcdupkKqlF',
                                    dest_path='/Data/AIIJC/aiijc_1578_goodFromTrain_pretrained.model')

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from question_processing import answering, qa_answering

from flask import Flask, request
from flask_cors import CORS
from pyngrok import ngrok, conf
import requests
import getpass
import json
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


In [7]:
url = ngrok.connect(5000, bind_tls=True).public_url
print(url)

https://3123-35-202-236-109.ngrok.io


In [8]:
requests.post('https://api.telegram.org/bot1957667602:AAGTgOj_a-tXF_MXldnD5kzt8Z6LjJ15Wiw/setWebhook', data={'url': url})

<Response [200]>

In [9]:
app = Flask(__name__)

os.environ["FLASK_ENV"] = "development"
app.config["BASE_URL"] = url

CORS(app)

hello = "Hey! I'm a bot which can answer on your question, but I can't answer on rhetorical or philosophical questions. Ask your question in English!"

accepted_letters = 'qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM'

def send_message(chat_id, text, reply_to_message_id):
    method = "sendMessage"
    token = "1957667602:AAGTgOj_a-tXF_MXldnD5kzt8Z6LjJ15Wiw"
    url = f"https://api.telegram.org/bot{token}/{method}"
    data = {"chat_id": chat_id, "text": text}
    if reply_to_message_id:
        data['reply_to_message_id'] = reply_to_message_id
    requests.post(url, data=data)

def find_bad_letter(text):
    for letter in text:
        if letter.isalpha() and letter not in accepted_letters:
            return True
    return False

@app.route("/", methods=["POST"])
def receive_update():
    if 'message' not in request.json:
        return {"ok": True}
    chat_id = request.json["message"]["chat"]["id"]
    text = request.json["message"]['text']
    if text in ['/start', '/restart']:
        send_message(chat_id, hello, None)
    else:
        if find_bad_letter(text):
            send_message(chat_id, "Sorry, but you can use only English letters", None)
            return {"ok": True}
        send_message(chat_id, "Please, wait a bit, and I'll answer", None)
        answer = answering(text)
        if answer == 'empty':
            answer = "I don't know, sorry"
        send_message(chat_id, answer, request.json["message"]['message_id'])
    return {"ok": True}

@app.route("/search-line", methods=["POST"])
def search_line():
    question = dict(json.loads(request.data))['question']
    if find_bad_letter(question):
        return {"status": "bad_letter"}
    else:
        answer = answering(question)
        if answer == 'empty':
            answer = "I don't know, sorry"
        return {"status": "OK", "answer": answer}

@app.route("/qa-search", methods=["POST"])
def qa_search():
    data = dict(json.loads(request.data))
    question = data['question']
    context = data['context']
    if find_bad_letter(question) or find_bad_letter(context):
        return {"status": "bad_letter"}
    else:
        answer = qa_answering(question, context)
        if answer == 'empty':
            answer = "I don't know, sorry"
        return {"status": "OK", "answer": answer}

In [ ]:
from gevent.pywsgi import WSGIServer

http_server = WSGIServer(('',5000), app)
http_server.serve_forever()